## This notebook inputs the missing values using KNN

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [2]:
#INPUT, OUTPUT = 'CompleteIndexes.csv','KNNImputed.csv'
INPUT, OUTPUT = 'CompleteWeeklyIndexes', 'KNNWeeklyImputed'

In [3]:
df = pd.read_csv(INPUT+'_training_FE.csv', parse_dates=True, index_col=0)
df_test = pd.read_csv(INPUT+'_test_FE.csv', parse_dates=True, index_col=0)

In [4]:
df_test.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,n100_obv_ema13,n100_obv_ema13_diff1_period3,n100_obv_ema26,n100_obv_ema26_diff1_period3,n100_obv_ema52,n100_obv_ema52_diff1_period3,n100_52_Week_High,n100_52_Week_High_diff1_period3,n100_52_Week_Low,n100_52_Week_Low_diff1_period3
2015-01-02,2058.199951,2.708700e+09,17.790001,0.0,17832.990234,76270000.0,4230.240234,1.435150e+09,NaN,NaN,...,3.100153e+09,3.190158e+08,2.466184e+09,2.296028e+08,1.262063e+09,1.834487e+08,NaN,NaN,NaN,NaN
2015-01-09,2044.810059,3.364140e+09,17.549999,0.0,17737.369141,93390000.0,4213.279785,1.715830e+09,17197.730469,155200.0,...,3.155977e+09,2.605298e+08,2.518610e+09,2.025883e+08,1.311490e+09,1.705574e+08,NaN,NaN,NaN,NaN
2015-01-16,2019.420044,4.056410e+09,20.950001,0.0,17511.570313,140480000.0,4142.140137,1.970520e+09,16864.160156,171100.0,...,3.250159e+09,2.485683e+08,2.591051e+09,1.985320e+08,1.371170e+09,1.690953e+08,NaN,NaN,NaN,NaN
2015-01-23,2051.820068,3.573560e+09,16.660000,0.0,17672.599609,97110000.0,4278.140137,1.653830e+09,17511.750000,132500.0,...,3.395105e+09,2.949520e+08,2.690620e+09,2.244360e+08,1.444910e+09,1.828471e+08,NaN,NaN,NaN,NaN
2015-01-30,1994.989990,4.568650e+09,20.969999,0.0,17164.949219,168560000.0,4148.430176,2.264230e+09,17674.390625,173500.0,...,3.490765e+09,3.347886e+08,2.766313e+09,2.477037e+08,1.506975e+09,1.954852e+08,NaN,NaN,NaN,NaN


In [5]:
#Scaled data has zero mean and unit variance
scaler = preprocessing.StandardScaler()
scaler.fit(df)
array = scaler.transform(df)
array_test = scaler.transform(df_test)
df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)
df_scaled.head()

C:\ProgramData\Anaconda2\envs\mestrado\lib\site-packages\sklearn\utils\extmath.py:770: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\ProgramData\Anaconda2\envs\mestrado\lib\site-packages\sklearn\utils\extmath.py:711: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,n100_obv_ema13,n100_obv_ema13_diff1_period3,n100_obv_ema26,n100_obv_ema26_diff1_period3,n100_obv_ema52,n100_obv_ema52_diff1_period3,n100_52_Week_High,n100_52_Week_High_diff1_period3,n100_52_Week_Low,n100_52_Week_Low_diff1_period3
2000-01-07,0.588334,-1.002211,0.099912,0.0,0.020227,-0.443957,1.707031,-0.537214,1.945019,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-14,0.676196,-1.085193,-0.125931,0.0,0.109104,0.336040,1.914791,-0.494165,2.196016,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-21,0.587926,-1.011385,0.001243,0.0,-0.099881,-0.244602,2.087059,0.035615,2.170210,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-28,0.286643,-1.079295,0.584489,0.0,-0.327300,-0.461189,1.608014,-0.574034,2.353305,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-04,0.524886,-1.109498,0.080178,0.0,-0.227555,-0.618274,2.116016,-0.306059,2.461300,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
imp = KNNImputer(n_neighbors=5, weights="uniform")
imp.fit(df_scaled)
df_imputed = imp.transform(df_scaled)
print(df_imputed.shape)
df_test_imputed = imp.transform(df_test_scaled)
df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)

(782, 370)


ValueError: Shape of passed values is (782, 370), indices imply (782, 372)

In [ ]:
df_final.tail()

In [ ]:
# Descaling the data
array_2 = scaler.inverse_transform(df_final)
df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

array_test_2 = scaler.inverse_transform(df_test_final)
df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)

df_final2.head()

In [ ]:
df_final2.to_csv(OUTPUT+'_training_FE.csv', index = True)
df_test_final2.to_csv(OUTPUT+ '_test_FE.csv', index = True)